In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

df_final = pd.read_csv("../data/df_final.csv")

In [4]:
cols = df.columns
print(df)

      year country_id_o  country_o country_id_d    country_d  contig  distcap  \
0     1965          SGP  Singapore          AFG  Afghanistan       0   5147.0   
1     1966          SGP  Singapore          AFG  Afghanistan       0   5147.0   
2     1967          SGP  Singapore          AFG  Afghanistan       0   5147.0   
3     1968          SGP  Singapore          AFG  Afghanistan       0   5147.0   
4     1969          SGP  Singapore          AFG  Afghanistan       0   5147.0   
...    ...          ...        ...          ...          ...     ...      ...   
7395  2016          SGP  Singapore          ZWE     Zimbabwe       0   8242.0   
7396  2017          SGP  Singapore          ZWE     Zimbabwe       0   8242.0   
7397  2018          SGP  Singapore          ZWE     Zimbabwe       0   8242.0   
7398  2019          SGP  Singapore          ZWE     Zimbabwe       0   8242.0   
7399  2020          SGP  Singapore          ZWE     Zimbabwe       0   8242.0   

        dist  comlang_off  

In [5]:
cols = ['year', 'country_id_o', 'country_o', 'country_id_d', 'country_d',
       'contig', 'distcap', 'dist', 'comlang_off', 'comlang_ethno', 'comrelig',
       'pop_o', 'pop_d', 'gdp_o', 'gdp_d', 'gdpcap_o', 'gdpcap_d', 'fta_wto',
       'tradeflow_comtrade_o', 'tradeflow_comtrade_d', 'tradeflow_imf_o',
       'tradeflow_imf_d', 'IdealPointDistance', 'arms', 'tot_o', 'tot_d',
       'tgdp_ratio_o', 'tgdp_ratio_d', 'avgtradeflow', 'logtradeflow',
       'v2x_polyarchy_o', 'v2x_libdem_o', 'v2x_partipdem_o', 'v2x_delibdem_o',
       'v2x_egaldem_o', 'v2x_polyarchy_d', 'v2x_libdem_d', 'v2x_partipdem_d',
       'v2x_delibdem_d', 'v2x_egaldem_d', 'allexports', 'chemicalexports',
       'consumerexports', 'foodexports', 'machineryexports',
       'manufacturesexports', 'logallexports', 'logchemicalexports',
       'logconsumerexports', 'logfoodexports', 'logmachineryexports',
       'logmanufacturesexports']

#identifier columns: year, country_id_d, country_d
I_cols = [cols[0]]+cols[3:5]

#columns of independent variables
X_cols = cols[5:18]+cols[22:28]+cols[30:40]
Y_cols = cols[28:30]+cols[40:]

X = df[X_cols]
Y = df[Y_cols]

# Standardize the data
X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X)
Y_scaler = StandardScaler()
Y_scaled = Y_scaler.fit_transform(Y)


df_X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
df_Y_scaled = pd.DataFrame(Y_scaled, columns=Y.columns, index=Y.index)
df_scaled = pd.concat([df[I_cols],df_X_scaled,df_Y_scaled],axis=1)

#print(df_scaled)
print(df_Y_scaled)

      avgtradeflow  logtradeflow  allexports  chemicalexports  \
0        -0.262860     -1.082182         NaN              NaN   
1        -0.261410     -0.558460         NaN              NaN   
2        -0.261340     -0.545624         NaN              NaN   
3        -0.262766     -1.011611         NaN              NaN   
4        -0.262926     -1.142885         NaN              NaN   
...            ...           ...         ...              ...   
7395     -0.193106      0.708362   -0.314363        -0.285664   
7396     -0.195322      0.697153   -0.314386        -0.285926   
7397     -0.181257      0.762857   -0.313881        -0.285595   
7398     -0.192640      0.710672   -0.314932        -0.285978   
7399     -0.231537      0.431265   -0.315012        -0.285951   

      consumerexports  foodexports  machineryexports  manufacturesexports  \
0                 NaN          NaN               NaN                  NaN   
1                 NaN          NaN               NaN             

In [6]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [7]:
target_var = 'logtradeflow'  # you can also try 'logtradeflow' or others
X_train, X_test, y_train, y_test = train_test_split(df_scaled[X_cols], df_scaled[target_var], test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': df_X_scaled.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize importances to sum to 1 (to use as weights)
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()

print(feature_importance_df)

               feature  importance    weight
9                gdp_d    0.429456  0.429456
4        comlang_ethno    0.083077  0.083077
1              distcap    0.082770  0.082770
3          comlang_off    0.073350  0.073350
12             fta_wto    0.041795  0.041795
11            gdpcap_d    0.036101  0.036101
2                 dist    0.031886  0.031886
18        tgdp_ratio_d    0.021159  0.021159
7                pop_d    0.019747  0.019747
25        v2x_libdem_d    0.019544  0.019544
5             comrelig    0.019118  0.019118
0               contig    0.017772  0.017772
24     v2x_polyarchy_d    0.014675  0.014675
23       v2x_egaldem_o    0.014390  0.014390
19     v2x_polyarchy_o    0.013572  0.013572
27      v2x_delibdem_d    0.013105  0.013105
28       v2x_egaldem_d    0.010641  0.010641
26     v2x_partipdem_d    0.010450  0.010450
16               tot_d    0.010420  0.010420
13  IdealPointDistance    0.008359  0.008359
21     v2x_partipdem_o    0.006148  0.006148
15        

In [8]:
def compute_index_row(row, weights, feature_names):
    total_weight = 0
    score = 0
    for feat, weight in zip(feature_names, weights):
        if pd.notnull(row[feat]):
            score += row[feat] * weight
            total_weight += weight
    return score / total_weight if total_weight > 0 else np.nan

In [9]:
# Ensure order of features in df_X_scaled matches the weights
feature_names = df_X_scaled.columns.tolist()
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values


In [10]:
df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

In [11]:
import statsmodels.api as sm

# Drop rows with NA in index or logtradeflow
ols_data = df[['advisability_index', 'logtradeflow']].dropna()

X_ols = ols_data[['advisability_index']]
y_ols = ols_data['logtradeflow']

# Add intercept for OLS
X_ols = sm.add_constant(X_ols)

ols_model = sm.OLS(y_ols, X_ols).fit()

print(ols_model.summary())
print(df)

                            OLS Regression Results                            
Dep. Variable:           logtradeflow   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     1378.
Date:                Sun, 06 Apr 2025   Prob (F-statistic):          1.02e-276
Time:                        07:07:55   Log-Likelihood:                -17653.
No. Observations:                7400   AIC:                         3.531e+04
Df Residuals:                    7398   BIC:                         3.532e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 17.5113      0

In [12]:
#predict logallexports using xgboost

df_scaled_new = df_scaled.dropna(subset=['logallexports'])

target_var = 'logallexports'  # you can also try 'logtradeflow' or others
X_train, X_test, y_train, y_test = train_test_split(df_scaled_new[X_cols], df_scaled_new[target_var], test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': df_X_scaled.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize importances to sum to 1 (to use as weights)
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()

print(feature_importance_df)

# Ensure order of features in df_X_scaled matches the weights
feature_names = df_X_scaled.columns.tolist()
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values

df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

import statsmodels.api as sm

# Drop rows with NA in index or logtradeflow
ols_data = df[['advisability_index', 'logallexports']].dropna()

X_ols = ols_data[['advisability_index']]
y_ols = ols_data['logallexports']

# Add intercept for OLS
X_ols = sm.add_constant(X_ols)

ols_model = sm.OLS(y_ols, X_ols).fit()

print(ols_model.summary())

               feature  importance    weight
9                gdp_d    0.412604  0.412604
1              distcap    0.099836  0.099836
4        comlang_ethno    0.093352  0.093352
11            gdpcap_d    0.062181  0.062181
2                 dist    0.037611  0.037611
12             fta_wto    0.031432  0.031432
3          comlang_off    0.030717  0.030717
7                pop_d    0.030474  0.030474
27      v2x_delibdem_d    0.028731  0.028731
17        tgdp_ratio_o    0.026799  0.026799
18        tgdp_ratio_d    0.024086  0.024086
5             comrelig    0.021781  0.021781
6                pop_o    0.017104  0.017104
25        v2x_libdem_d    0.015031  0.015031
28       v2x_egaldem_d    0.012140  0.012140
24     v2x_polyarchy_d    0.011539  0.011539
26     v2x_partipdem_d    0.009030  0.009030
13  IdealPointDistance    0.007176  0.007176
16               tot_d    0.007021  0.007021
0               contig    0.005887  0.005887
8                gdp_o    0.003904  0.003904
19     v2x

In [61]:
#predict logconsumerexports using xgboost

df_scaled_new = df_scaled.dropna(subset=['logconsumerexports'])

target_var = 'logconsumerexports'  # you can also try 'logtradeflow' or others
X_train, X_test, y_train, y_test = train_test_split(df_scaled_new[X_cols], df_scaled_new[target_var], test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': df_X_scaled.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize importances to sum to 1 (to use as weights)
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()

print(feature_importance_df)

# Ensure order of features in df_X_scaled matches the weights
feature_names = df_X_scaled.columns.tolist()
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values

df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

import statsmodels.api as sm

# Drop rows with NA in index or logtradeflow
ols_data = df[['advisability_index', 'logconsumerexports']].dropna()

X_ols = ols_data[['advisability_index']]
y_ols = ols_data['logconsumerexports']

# Add intercept for OLS
X_ols = sm.add_constant(X_ols)

ols_model = sm.OLS(y_ols, X_ols).fit()

print(ols_model.summary())

               feature  importance    weight
9                gdp_d    0.274429  0.274429
1              distcap    0.100367  0.100367
4        comlang_ethno    0.099198  0.099198
2                 dist    0.096146  0.096146
12             fta_wto    0.076804  0.076804
7                pop_d    0.049417  0.049417
11            gdpcap_d    0.030293  0.030293
5             comrelig    0.028877  0.028877
6                pop_o    0.026053  0.026053
25        v2x_libdem_d    0.023522  0.023522
13  IdealPointDistance    0.020306  0.020306
27      v2x_delibdem_d    0.019807  0.019807
26     v2x_partipdem_d    0.019443  0.019443
3          comlang_off    0.018512  0.018512
28       v2x_egaldem_d    0.018445  0.018445
24     v2x_polyarchy_d    0.015158  0.015158
18        tgdp_ratio_d    0.014457  0.014457
17        tgdp_ratio_o    0.014289  0.014289
16               tot_d    0.012743  0.012743
10            gdpcap_o    0.010088  0.010088
19     v2x_polyarchy_o    0.007292  0.007292
15        

In [62]:
#predict logchemicalexports using xgboost

df_scaled_new = df_scaled.dropna(subset=['logchemicalexports'])

target_var = 'logchemicalexports'  # you can also try 'logtradeflow' or others
X_train, X_test, y_train, y_test = train_test_split(df_scaled_new[X_cols], df_scaled_new[target_var], test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': df_X_scaled.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize importances to sum to 1 (to use as weights)
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()

print(feature_importance_df)

# Ensure order of features in df_X_scaled matches the weights
feature_names = df_X_scaled.columns.tolist()
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values

df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

import statsmodels.api as sm

# Drop rows with NA in index or logtradeflow
ols_data = df[['advisability_index', 'logchemicalexports']].dropna()

X_ols = ols_data[['advisability_index']]
y_ols = ols_data['logchemicalexports']

# Add intercept for OLS
X_ols = sm.add_constant(X_ols)

ols_model = sm.OLS(y_ols, X_ols).fit()

print(ols_model.summary())

               feature  importance    weight
9                gdp_d    0.350086  0.350086
1              distcap    0.105723  0.105723
4        comlang_ethno    0.094755  0.094755
12             fta_wto    0.081539  0.081539
2                 dist    0.039794  0.039794
7                pop_d    0.035716  0.035716
5             comrelig    0.029313  0.029313
11            gdpcap_d    0.028809  0.028809
21     v2x_partipdem_o    0.027905  0.027905
19     v2x_polyarchy_o    0.021666  0.021666
18        tgdp_ratio_d    0.021202  0.021202
3          comlang_off    0.020674  0.020674
25        v2x_libdem_d    0.016685  0.016685
28       v2x_egaldem_d    0.016451  0.016451
15               tot_o    0.013404  0.013404
26     v2x_partipdem_d    0.013194  0.013194
16               tot_d    0.012995  0.012995
27      v2x_delibdem_d    0.012595  0.012595
24     v2x_polyarchy_d    0.011829  0.011829
6                pop_o    0.011196  0.011196
13  IdealPointDistance    0.008579  0.008579
17        

In [63]:
#predict logfoodexports using xgboost

df_scaled_new = df_scaled.dropna(subset=['logfoodexports'])

target_var = 'logfoodexports'  # you can also try 'logtradeflow' or others
X_train, X_test, y_train, y_test = train_test_split(df_scaled_new[X_cols], df_scaled_new[target_var], test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': df_X_scaled.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize importances to sum to 1 (to use as weights)
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()

print(feature_importance_df)

# Ensure order of features in df_X_scaled matches the weights
feature_names = df_X_scaled.columns.tolist()
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values

df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

import statsmodels.api as sm

# Drop rows with NA in index or logtradeflow
ols_data = df[['advisability_index', 'logfoodexports']].dropna()

X_ols = ols_data[['advisability_index']]
y_ols = ols_data['logfoodexports']

# Add intercept for OLS
X_ols = sm.add_constant(X_ols)

ols_model = sm.OLS(y_ols, X_ols).fit()

print(ols_model.summary())

               feature  importance    weight
12             fta_wto    0.331823  0.331823
9                gdp_d    0.103140  0.103140
1              distcap    0.090644  0.090644
4        comlang_ethno    0.075600  0.075600
11            gdpcap_d    0.038588  0.038588
2                 dist    0.035393  0.035393
21     v2x_partipdem_o    0.030469  0.030469
20        v2x_libdem_o    0.026769  0.026769
3          comlang_off    0.025384  0.025384
7                pop_d    0.024830  0.024830
23       v2x_egaldem_o    0.021832  0.021832
22      v2x_delibdem_o    0.020887  0.020887
26     v2x_partipdem_d    0.020351  0.020351
5             comrelig    0.019090  0.019090
25        v2x_libdem_d    0.016497  0.016497
24     v2x_polyarchy_d    0.016212  0.016212
17        tgdp_ratio_o    0.013569  0.013569
6                pop_o    0.012513  0.012513
13  IdealPointDistance    0.011539  0.011539
27      v2x_delibdem_d    0.010343  0.010343
16               tot_d    0.010238  0.010238
18        

In [64]:
#predict logmachineryexports using xgboost

df_scaled_new = df_scaled.dropna(subset=['logmachineryexports'])

target_var = 'logmachineryexports'  # you can also try 'logtradeflow' or others
X_train, X_test, y_train, y_test = train_test_split(df_scaled_new[X_cols], df_scaled_new[target_var], test_size=0.2, random_state=42)

model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
model.fit(X_train, y_train)


importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': df_X_scaled.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize importances to sum to 1 (to use as weights)
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()

print(feature_importance_df)

# Ensure order of features in df_X_scaled matches the weights
feature_names = df_X_scaled.columns.tolist()
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values

df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

import statsmodels.api as sm

# Drop rows with NA in index or logtradeflow
ols_data = df[['advisability_index', 'logmachineryexports']].dropna()

X_ols = ols_data[['advisability_index']]
y_ols = ols_data['logmachineryexports']

# Add intercept for OLS
X_ols = sm.add_constant(X_ols)

ols_model = sm.OLS(y_ols, X_ols).fit()

print(ols_model.summary())

               feature  importance    weight
9                gdp_d    0.431951  0.431951
4        comlang_ethno    0.090292  0.090292
1              distcap    0.088999  0.088999
0               contig    0.037705  0.037705
27      v2x_delibdem_d    0.025777  0.025777
5             comrelig    0.024387  0.024387
18        tgdp_ratio_d    0.024268  0.024268
2                 dist    0.023986  0.023986
7                pop_d    0.023021  0.023021
12             fta_wto    0.022032  0.022032
11            gdpcap_d    0.021901  0.021901
3          comlang_off    0.020682  0.020682
6                pop_o    0.020439  0.020439
8                gdp_o    0.019309  0.019309
26     v2x_partipdem_d    0.017271  0.017271
24     v2x_polyarchy_d    0.015356  0.015356
21     v2x_partipdem_o    0.013861  0.013861
25        v2x_libdem_d    0.012655  0.012655
28       v2x_egaldem_d    0.012604  0.012604
19     v2x_polyarchy_o    0.010836  0.010836
13  IdealPointDistance    0.009572  0.009572
15        

In [60]:
#random forest

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import statsmodels.api as sm

target_var = 'logtradeflow'  # or 'logtradeflow'
X_train, X_test, y_train, y_test = train_test_split(df_scaled[X_cols], df_scaled[target_var], test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=200, max_depth=6, random_state=42)
rf_model.fit(X_train, y_train)

importances = rf_model.feature_importances_
feature_names = df_X_scaled.columns.tolist()

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Normalize to get weights
feature_importance_df['weight'] = feature_importance_df['importance'] / feature_importance_df['importance'].sum()
print(feature_importance_df)

def compute_index_row(row, weights, feature_names):
    total_weight = 0
    score = 0
    for feat, weight in zip(feature_names, weights):
        if pd.notnull(row[feat]):
            score += row[feat] * weight
            total_weight += weight
    return score / total_weight if total_weight > 0 else np.nan

# Prepare weights
weights_vector = feature_importance_df.set_index('feature').loc[feature_names, 'weight'].values


# Compute row-wise index
df['advisability_index'] = df_scaled.apply(lambda row: compute_index_row(row, weights_vector, feature_names), axis=1)

ols_data = df[['advisability_index', 'logtradeflow']].dropna()
X_ols = sm.add_constant(ols_data[['advisability_index']])
y_ols = ols_data['logtradeflow']

ols_model = sm.OLS(y_ols, X_ols).fit()
print(ols_model.summary())


               feature  importance    weight
9                gdp_d    0.690485  0.690485
1              distcap    0.104888  0.104888
2                 dist    0.093297  0.093297
3          comlang_off    0.019838  0.019838
18        tgdp_ratio_d    0.018583  0.018583
11            gdpcap_d    0.018219  0.018219
7                pop_d    0.018142  0.018142
5             comrelig    0.011049  0.011049
24     v2x_polyarchy_d    0.004516  0.004516
16               tot_d    0.003892  0.003892
4        comlang_ethno    0.003045  0.003045
28       v2x_egaldem_d    0.002815  0.002815
12             fta_wto    0.002331  0.002331
26     v2x_partipdem_d    0.001536  0.001536
25        v2x_libdem_d    0.001462  0.001462
13  IdealPointDistance    0.001448  0.001448
15               tot_o    0.001314  0.001314
27      v2x_delibdem_d    0.000767  0.000767
8                gdp_o    0.000514  0.000514
6                pop_o    0.000403  0.000403
10            gdpcap_o    0.000306  0.000306
17        